In [ ]:
import torch
import json
from torch.utils.data import Dataset
import numpy as np
from transformers import AutoTokenizer, RobertaTokenizerFast

In [ ]:
all_questions = {
    "COURT": "Which court is mentioned in the text?",
    "DATE": "What is the date mentioned in the text?",
    # "PETITIONER": "Who is the petitioner mentioned in the text?",
    # "RESPONDENT": "Who is the respondent mentioned in the text?",
    "JUDGE": "Who is the judge mentioned in the text?",
    # "ORG": "What organization is mentioned in the text?",
    # "GPE": "What geopolitical entity is mentioned in the text?",
    # "STATUTE": "What statute is mentioned in the text?",
    # "PROVISION": "What provision is mentioned in the text?",
    # "PRECEDENT": "What precedent is mentioned in the text?",
    # "CASE_NUMBER": "What is the case number mentioned in the text?",
    # "WITNESS": "Who is the witness mentioned in the text?",
    # "OTHER_PERSON": "Who is the other person mentioned in the text?",
    # "LAWYER": "Who is the lawyer mentioned in the text?"
}

In [ ]:
class LegalQADataSet:
    def __init__(self, dataset_path):
        self.data = json.load(open(dataset_path))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["data"]["text"]

        qa_pairs = []
        # generate all
        for tag, question in all_questions.items():
            qa_pair = {"context": text, "question": question, "tag": tag}
            qa_pairs.append(qa_pair)

        # Generate a question-answer pair from annotations
        labels = []
        for annotation in item["annotations"][0]["result"]:
            label = annotation["value"]["labels"][0]
            answer_start = annotation["value"]["start"]
            answer_end = annotation["value"]["end"]
            if label in all_questions.keys():
                labels.append(
                    {"label": label, "start": answer_start, "end": answer_end}
                )

        return {"qa_pairs": qa_pairs, "labels": labels}

In [ ]:
from transformers import pipeline
import torch

# Create a QA pipeline
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
qa_pipeline = pipeline(
    "question-answering", model="deepset/roberta-base-squad2", device=device
)  # 指定device=0表示使用第一个GPU

dataset_path = (
    "/kaggle/working/NER_DEV/NER_DEV/NER_DEV_ALL.json"
)
dataset = LegalQADataSet(dataset_path)

In [ ]:
import torch
from sklearn.metrics import f1_score
from tqdm import tqdm


# Store real and predicted labels
true_labels = []
predicted_labels = []


# Use pipeline for inference and record predicted labels
for sample in tqdm(dataset):
    # For each QA pair, get the context, question, and ground truth label
    true_labels.append(sample["labels"])
    output_labels = []
    for qa_pair in sample["qa_pairs"]:
        context = qa_pair["context"]
        question = qa_pair["question"]
        tag = qa_pair["tag"]

        # print(question)

        # use pipeline to predict
        qa_input = {"question": question, "context": context}
        qa_output = qa_pipeline(qa_input)
        ans_confidence = qa_output["score"]
        if ans_confidence >= 0.2:
            output_label = {
                "label": tag,
                "start": qa_output["start"],
                "end": qa_output["end"],
            }
            output_labels.append(output_label)
    predicted_labels.append(output_labels)

results = [
    {"predicted": p_l, "true": t_l} for p_l, t_l in zip(predicted_labels, true_labels)
]

with open("qa_ner.json", "w") as fp:
    json.dump(results, fp)

In [ ]:
from nervaluate import Evaluator

evaluator = Evaluator(true_labels, predicted_labels, tags=all_questions.keys())
results, results_per_tag = evaluator.evaluate()

with open("qa_ner_evaluation.json", "w") as fp:
    json.dump([results, results_per_tag], fp)